<a href="https://colab.research.google.com/github/mkbahk/AmazonBraket/blob/main/UnitaryMatrixtoQuantumOperator_Gates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install qiskit[visualization]==1.2.4
!python -m pip install qiskit-aer==0.15.1
!python -m pip install qiskit-ionq==0.5.8

In [ ]:
#import IonQ backends
from qiskit_ionq import IonQProvider
ionq_provider = IonQProvider(token='4JZX9TXxZ5P0ymsPdpKDpxozbrNSLzxF')

#Import other qiskit requirements
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, array_to_latex #Vector Martix를 좀더 fancy하게 표현하기 위함
from qiskit.circuit.library import UnitaryGate #사용자 정의 양자게이트를 만든는 용도, qiskit.extensions에서 qiskit.circuit.library로 변경됨

**Qustion 1: Implementing CNOT (Simulator vs. Hardware)**


In [ ]:
qc1 = QuantumCircuit(2, 2) #Instantiate a circuit with two qubits and two classical bits
qc1.cx(0, 1) #Apply the CNOT gate with control qubit 0 and target qubit 1

# Measure all the qubits
for i in range(len(qc1.qubits)):
  qc1.measure(i, i)
###for

#Draw the circuit to visualize it
qc1.draw("mpl")

In [ ]:
aer_backend = Aer.get_backend('qasm_simulator') #choose your backend
job = aer_backend.run(qc1, shots=1024) #create the job object
result = job.result() #create the result object
counts = result.get_counts() #get the counts dictionary
plot_histogram(counts) #plot the histogram of counts

In [ ]:
ionq_backend = ionq_provider.get_backend('ionq_simulator') #choose your backend
ionq_backend

In [ ]:
from qiskit import transpile
# Transpile the circuit for the IonQ backend
# This is the crucial step to resolve the error
transpiled_qc = transpile(qc1, backend=ionq_backend, optimization_level=3)

In [ ]:
job = ionq_backend.run(transpiled_qc, shots=100) #create the job object

# Check if job is complete
from qiskit.providers.jobstatus import JobStatus
import time
while job.status() is not JobStatus.DONE:
  print("Job Status is ", job.status())
  time.sleep(5)
###while

result = job.result() #create the result object
counts = result.get_counts() #get the counts dictionary
plot_histogram(counts) #plot the histogram of counts

In [ ]:
# First we will create our very own, new quantum gate

my_circuit = QuantumCircuit(2) # Our gate is a two-qubit gate, so initialize a two-qubit quantum circuit
my_circuit.x(0) # our gate has a NOT gate
my_circuit.cx(0, 1)# our gate then has a CNOT gate
my_circuit.x(0) # then we have a NOT Gate
my_circuit.draw("mpl") #Visualization

In [ ]:
#convert your circuit to a gate
xcx = my_circuit.to_gate(label="XCX")
type(xcx)

In [ ]:
qc2 = QuantumCircuit(2, 2) #Instantiate a circuit with two qubits and two clasical bits
qc2.append(xcx, [0, 1])

# Measure all the qubits
for i in range(len(qc2.qubits)):
  qc2.measure(i, i)
###for

qc2.draw("mpl")

In [ ]:
from qiskit import transpile
# Transpile the circuit for the IonQ backend
# This is the crucial step to resolve the error
transpiled2_qc = transpile(qc2, backend=ionq_backend, optimization_level=3)

In [ ]:
job = ionq_backend.run(transpiled_qc, shots=100) #create the job object


# Check if job is complete
from qiskit.providers.jobstatus import JobStatus
import time
while job.status() is not JobStatus.DONE:
  print("Job Status is ", job.status())
  time.sleep(5)
###while

result = job.result() #create the result object
counts = result.get_counts() #get the counts
plot_histogram(counts)

In [ ]:
#First we will create the regular V-gate
matrix = [[0.5 * (1 - 1j), 0.5 * (1 - 1j) * 1j],
          [0.5 * (1 - 1j) * 1j, 0.5 * (1 - 1j)]] #enter the marix the V-gate
v = UnitaryGate(matrix, 'V') #Defining V
#Trying out our V-gate
qc3 = QuantumCircuit(1) #Initialise Quantum Circuit
qc3.unitary(v, 0) #Act unitary on circuit
print(qc3.draw()) #Visualize circuit
#matrix of V-gate
aer_backend = Aer.get_backend('unitary_simulator') #use unitary_simulator as backend
job = aer_backend.run(qc3, shots=1024) #create job object
result = job.result() #create result object
matrix = result.get_unitary() #get the unitary martix corresponding to the circuit
array_to_latex(matrix)#illustrate the matrix

In [ ]:
controlled_v = v.control() #define the controlled-v operation
# Trying out the controlled-v gate
qc4 = QuantumCircuit(2) #Define a Quantum Circuit
qc4.append(controlled_v, [0, 1]) #Append the controlled_v gate
qc4.draw("mpl") #Visualize

In [ ]:
v_dagger = v.adjoint() #define v-dagger
controlled_v_dagger = v_dagger.control() #define the controlled-v-dagger

#Trying it out
qc5 = QuantumCircuit(1) #Define a quantum circuit
qc5.unitary(v_dagger, 0) #implement v-dagger
print(qc5.draw()) #implement v-dagger

#matrix of V-dagger
aer_unitary_backend = Aer.get_backend('unitary_simulator') #use unitary_simulator as backend
job = aer_unitary_backend.run(qc5, shots=1024) #create job object
result = job.result() #create result object
matrix = result.get_unitary() #get the unitary martix corresponding to the circuit
array_to_latex(matrix)#illustrate the matrix


In [ ]:
qc7 = QuantumCircuit(3) # A Toffoli gate is a 3-qubit gate. So define a 3-qubit quantum circuit
qc7.append(controlled_v, [1, 2]) # Append your controlled-V operation
qc7.cx(0, 1) # CNOT
qc7.append(controlled_v_dagger, [1, 2]) # Controlled-V dagger
qc7.cx(0, 1) # CNOT
qc7.append(controlled_v, [0, 2]) # One more controole-V
qc7.draw("mpl") # Visualize

In [ ]:
qc7.decompose().draw("mpl")

In [ ]:
qc7.decompose().decompose().draw("mpl")

In [ ]:
qc7.decompose().decompose().decompose().draw("mpl")

In [ ]:
qc7.decompose().decompose().decompose().decompose().draw("mpl")

In [ ]:
#End of Codes